---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [15]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [90]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

P1_Graphs is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
- Preferential Attachment ('PA')
- Small World with low probability of rewiring ('SW_L')
- Small World with high probability of rewiring ('SW_H')

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

The graph_identification function should return a list of length 5 where each element in the list is either 'PA', 'SW_L', or 'SW_H'.

In [91]:
# import matplotlib.pyplot as plt
# for G in P1_Graphs:
#     degrees = G.degree()
#     degree_values = sorted(set(degrees.values()))
#     histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
#     plt.bar(degree_values, histogram)
#     plt.xlabel('Degree')
#     plt.ylabel('Fraction of Nodes')
#     plt.show()

In [92]:
# import matplotlib.pyplot as plt
# for G in P1_Graphs:
#     degrees = G.degree()
#     degree_values = sorted(set(degrees.values()))
#     histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
#     plt.plot(degree_values, histogram, 'o')
#     plt.xlabel('Degree')
#     plt.ylabel('Fraction of Nodes')
#     plt.xscale('log')
#     plt.yscale('log')
#     plt.show()


In [93]:
# print('average path length')
# for G in P1_Graphs:
#     print(nx.average_shortest_path_length(G))
# print('clustering coefficient')
# for G in P1_Graphs:
#     print(nx.average_clustering(G))

In [94]:
# for G in P1_Graphs:
#     degrees = G.degree()
#     degree_values = sorted(set(degrees.values()))
#     histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
#     print(histogram)
#     print(len(histogram))

In [95]:
# for G in P1_Graphs:
#     print(nx.info(G))
#     print('\n')

In [96]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']


# import hashlib
# hashlib.sha256(str(graph_identification()).encode('utf8')).hexdigest() == '47b65e3e18ba3808079ab3bb43d76ba29a3a0f8f8930cd51db55bbc295427b52'

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [16]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))
print('\n')
print(G.nodes(data=True))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


[(0, {'Department': 1, 'ManagementSalary': 0.0}), (1, {'Department': 1, 'ManagementSalary': nan}), (2, {'Department': 21, 'ManagementSalary': nan}), (3, {'Department': 21, 'ManagementSalary': 1.0}), (4, {'Department': 21, 'ManagementSalary': 1.0}), (5, {'Department': 25, 'ManagementSalary': nan}), (6, {'Department': 25, 'ManagementSalary': 1.0}), (7, {'Department': 14, 'ManagementSalary': 0.0}), (8, {'Department': 14, 'ManagementSalary': nan}), (9, {'Department': 14, 'ManagementSalary': 0.0}), (10, {'Department': 9, 'ManagementSalary': 0.0}), (11, {'Department': 14, 'ManagementSalary': 0.0}), (12, {'Department': 14, 'ManagementSalary': 1.0}), (13, {'Department': 26, 'ManagementSalary': 1.0}), (14, {'Department': 4, 'ManagementSalary': nan}), (15, {'Department': 17, 'ManagementSalary': 0.0}), (16, {'Department': 34, 'ManagementSalary': 0.0}), (17, {'Department': 1, 'ManagementSalary': 0.0}), (18, 

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc

def salary_predictions():
    # -------- calculate several features
    degrees = pd.Series(nx.degree(G))
    degCent = pd.Series(nx.degree_centrality(G))
    closeCent = pd.Series(nx.degree_centrality(G))
    betCent = pd.Series(nx.betweenness_centrality(G))
    clustering = pd.Series(nx.clustering(G))
    
    # -------- convert networkx graph to a dataframe
    df = pd.DataFrame(index=G.nodes())
    df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['management_salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    # -------- add centrality features to df
    df['degree'], df['degCent'], df['closeCent'], df['betCent'], df['clustering'] = degrees, degCent, closeCent, betCent, clustering

    # -------- split training set and target set
    target = df[df['management_salary'].isnull()]
    training = df[~df['management_salary'].isnull()] 
    
    # -------- normalize data and train a mlp model
    scaler = MinMaxScaler()
    y_train = training[['management_salary']]
    X = training.drop(['management_salary'], axis=1)
    X_train_scaled = scaler.fit_transform(X)
    target_scaled = scaler.transform(target[X.columns])
    
    mlp = MLPClassifier().fit(X_train_scaled, y_train)
    
    # -------- predict target set
    predictions = mlp.predict_proba(target_scaled)[:,1]

    
    return pd.Series(predictions, index=target.index)
salary_predictions()

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:904: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1       0.100868
2       0.673307
5       0.992732
8       0.110921
14      0.307755
18      0.157808
27      0.272312
30      0.300598
31      0.151350
34      0.137282
37      0.099006
40      0.203736
45      0.161511
54      0.246555
55      0.238155
60      0.169557
62      0.997207
65      0.634148
77      0.053701
79      0.085632
97      0.096457
101     0.057619
103     0.159286
108     0.093659
113     0.459035
122     0.041443
141     0.466115
142     0.928961
144     0.098105
145     0.457517
          ...   
913     0.052935
914     0.081152
915     0.009774
918     0.114899
923     0.055355
926     0.100026
931     0.064860
934     0.029485
939     0.017233
944     0.008050
945     0.053191
947     0.160377
950     0.107511
951     0.057243
953     0.031579
959     0.011332
962     0.010417
963     0.126591
968     0.150318
969     0.140548
974     0.082020
984     0.028397
987     0.128726
989     0.132774
991     0.202929
992     0.011832
994     0.008196
996     0.0091

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    feature_lst = ['comm_neighbors', 'jaccard_coefficient', 'resource_allocation', 'adamicadar_index', 'pref_attachment']
    features = pd.DataFrame()
    non_edges = nx.non_edges(G)
    
    neigh_dict, jac_dict, res_dict, adam_dict, pref_dict = {}, {}, {}, {}, {}
    for pair in non_edges:
        neigh_dict[pair] = len(list(nx.common_neighbors(G, pair[0], pair[1])))
        
    for u,v,p in nx.jaccard_coefficient(G):
        jac_dict[(u,v)] = p
    
    for u,v,p in nx.resource_allocation_index(G):
        res_dict[(u,v)] = p
    
    for u,v,p in nx.adamic_adar_index(G):
        adam_dict[(u,v)] = p
    
    for u,v,p in nx.preferential_attachment(G):
        pref_dict[(u,v)] = p
    
    
    features['edges'] = neigh_dict.keys()
    features['comm_neighbors'] = features['edges'].map(neigh_dict)
    features['jaccard_coefficient'] = features['edges'].map(jac_dict)
    features['resource_allocation'] = features['edges'].map(res_dict)
    features['adamicadar_index'] = features['edges'].map(adam_dict)
    features['pref_attachemnt'] = features['edges'].map(pref_dict)
#     print(features.head())
    
    full = pd.merge(future_connections.reset_index(), features, how='left', left_on='index', right_on='edges').set_index('edges').drop(['index'], axis=1)
#     print(full.head())

    training = full[~full['Future Connection'].isnull()]
    target = full[full['Future Connection'].isnull()]
    
    scaler = MinMaxScaler()
    y_train = training[['Future Connection']]
    X = training.drop(['Future Connection'], axis=1)
#     print(type(X))
#     print(X.head())
    X_train_scaled = scaler.fit_transform(X)
    target_scaled = scaler.transform(target[X.columns])
    
    mlp = MLPClassifier().fit(X_train_scaled, y_train)

    predictions = mlp.predict_proba(target_scaled)[:,1]

    
    return pd.Series(predictions, index=target.index)

new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:904: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
